#### 同策略基金分析

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from pyecharts.charts import Tab
from pyecharts.components import Table
from function import *
from WindPy import w
from pathlib import Path

w.start()

import warnings

warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")
plt.rcParams["font.sans-serif"] = ["Microsoft YaHei"]
plt.rcParams["axes.unicode_minus"] = False

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2024 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


In [2]:
# 辅助日期
year_begin_date = "2023-12-29"
month_begin_date = "2024-11-29"
week_begin_date = "2024-12-13"

In [3]:
basic_info = load_data("产品目录.xlsx")
basic_info

,大类策略,细分策略,基金代码,基金名称,基准代码,基准名称,最大回撤阈值
0,主观CTA,复合,SM6151,草本优益1号,NH0100.NHF,南华商品指数,-0.05
1,主观CTA,复合,SM6151,草本致远1号,NH0300.NHF,南华农产品指数,-0.10
2,主观CTA,趋势,SGC034,济海实恒对冲二号,NH0100.NHF,南华商品指数,-0.10
3,主观CTA,复合,ST6571,星恒哲萌1号,NH0100.NHF,南华商品指数,-0.05
4,量化CTA,截面,SS0221,均成CTA1号,NH0100.NHF,南华商品指数,-0.10
5,量化CTA,趋势,SXF557,千衍九凌1号,NH0100.NHF,南华商品指数,-0.03
6,量化CTA,趋势,SCV067,众壹资产量合兴成一号,NH0100.NHF,南华商品指数,-0.05
7,量化CTA,趋势,SLS182,弘源多元化CTA,NH0100.NHF,南华商品指数,-0.05
8,量化CTA,趋势,SN3221,宏锡量化CTA7号,NH0100.NHF,南华商品指数,-0.05
9,量化CTA,复合,SGA623,会世元丰CTA1号,NH0100.NHF,南华商品指数,-0.10


#### 表格

In [4]:
nav_dfs = None
nav_table = pd.DataFrame(
    columns=[
        "大类策略",
        "细分策略",
        "产品名称",
        "日期",
        "单位净值",
        "累计净值",
        "复权净值",
        "本年收益(%)",
        "当月收益(%)",
        "当周收益(%)",
    ]
)
for fund in basic_info["基金名称"]:
    # 导入净值数据
    files_list_series = pd.Series([i.stem for i in Path("./data").glob("*.xlsx")])
    nav_df_path = Path("data").joinpath(f"{files_list_series[files_list_series.str.contains(fund)].item()}.xlsx")
    nav_df = load_data(nav_df_path)
    nav_df = get_standardized_data(nav_df)
    nav_df.rename(
        columns={
            "date": "日期",
            "nav_unit": "单位净值",
            "nav_accumulated": "累计净值",
            "nav_adjusted": "复权净值",
        },
        inplace=True,
    )
    # 计算年、月、周收益
    year_begin_nav = nav_df[nav_df["日期"] == year_begin_date]
    month_begin_nav = nav_df[nav_df["日期"] == month_begin_date]
    week_begin_nav = nav_df[nav_df["日期"] == week_begin_date]
    nav_head = nav_df.head(1)
    nav_tail = nav_df.tail(1)
    nav_tail["产品名称"] = fund
    nav_tail["大类策略"] = basic_info[basic_info["基金名称"] == fund]["大类策略"].item()
    nav_tail["细分策略"] = basic_info[basic_info["基金名称"] == fund]["细分策略"].item()
    if not year_begin_nav.empty:
        year_begin_value = year_begin_nav["复权净值"].iloc[0]
        nav_tail["本年收益(%)"] = round(
            ((nav_tail["复权净值"].iloc[0] / year_begin_value) - 1) * 100, 2
        )
    else:  # 如果找不到2023-12-29净值，则使用第一条数据
        nav_tail["本年收益(%)"] = round(
            ((nav_tail["复权净值"].iloc[0] / nav_head["复权净值"].iloc[0]) - 1) * 100, 2
        )
    month_begin_value = month_begin_nav["复权净值"].iloc[0]
    nav_tail["当月收益(%)"] = round(
        ((nav_tail["复权净值"].iloc[0] / month_begin_value) - 1) * 100, 2
    )
    week_begin_value = week_begin_nav["复权净值"].iloc[0]
    nav_tail["当周收益(%)"] = round(
        ((nav_tail["复权净值"].iloc[0] / week_begin_value) - 1) * 100, 2
    )
    # 添加至表格
    nav_table = pd.concat([nav_table, nav_tail], ignore_index=True)
    custom_order = ["主观CTA", '量化CTA', "套利"]
    custom_order_ = ["趋势", '截面', "复合","套利"]
    nav_table['大类策略'] = pd.Categorical(nav_table['大类策略'], categories=custom_order, ordered=True)
    nav_table['细分策略'] = pd.Categorical(nav_table['细分策略'], categories=custom_order_, ordered=True)
    nav_table.sort_values(by=['大类策略', "细分策略", '本年收益(%)'], inplace=True, ascending=[True, True, False], ignore_index=True)
nav_table["日期"] = nav_table["日期"].map(lambda x: x.strftime("%Y-%m-%d"))

In [5]:
table_index = Table()
headers = list(nav_table.columns)
rows = nav_table.values
table_index.add(headers, rows)

In [6]:
nav_table

,大类策略,细分策略,产品名称,日期,单位净值,累计净值,复权净值,本年收益(%),当月收益(%),当周收益(%)
0,主观CTA,趋势,济海实恒对冲二号,2024-12-20,1.0520,1.9480,2.2411,11.51,-0.28,-0.75
1,主观CTA,复合,草本致远1号,2024-12-20,1.0370,3.0100,4.3253,49.01,-1.89,-2.54
2,主观CTA,复合,草本优益1号,2024-12-20,0.9940,3.5560,5.4755,16.72,-3.97,-1.39
3,主观CTA,复合,星恒哲萌1号,2024-12-20,1.1133,1.5766,1.6899,12.04,0.36,0.17
4,量化CTA,趋势,旭诺CTA三号,2024-12-20,3.7666,3.7666,3.7666,65.59,1.90,2.59
5,量化CTA,趋势,千衍九凌1号,2024-12-20,1.3131,1.7841,1.3131,31.31,2.53,0.53
6,量化CTA,趋势,弘源多元化CTA,2024-12-20,1.6600,1.6600,1.6600,15.76,1.78,1.41
7,量化CTA,趋势,宏锡量化CTA7号,2024-12-20,1.2783,2.2005,2.8137,12.85,1.85,2.33
8,量化CTA,趋势,众壹资产量合兴成一号,2024-12-20,1.2360,1.5080,1.2612,11.05,0.33,1.64
9,量化CTA,截面,均成CTA1号,2024-12-20,6.5990,7.2440,7.5888,6.02,-1.18,-0.48


#### 折线图

In [7]:
def get_data(strategy):
    strategy_info = basic_info.loc[(basic_info["大类策略"] == strategy), :]
    strategy_info.reset_index(drop=True, inplace=True)
    nav_dfs = None
    for fund in strategy_info["基金名称"]:
        files_list_series = pd.Series([i.stem for i in Path("./data").glob("*.xlsx")])
        nav_df_path = Path("data").joinpath(f"{files_list_series[files_list_series.str.contains(fund)].item()}.xlsx")
        nav_df = load_data(nav_df_path)
        nav_df = get_standardized_data(nav_df)
        nav_df.rename(columns={"date": "日期", "nav_adjusted": fund}, inplace=True)
        nav_df = nav_df[["日期", fund]]
        if nav_dfs is None:
            nav_dfs = nav_df
        else:
            nav_dfs = pd.merge(nav_dfs, nav_df, on="日期", how="inner")
            nav_dfs = nav_dfs.set_index("日期")
            nav_dfs = nav_dfs.loc["2023-12-29":]
            nav_dfs = nav_dfs.div(nav_dfs.iloc[0])
    return nav_dfs

In [8]:
get_line(get_data("主观CTA"), "主观CTA").render_notebook()

In [9]:
get_line(get_data("量化CTA"), "量化CTA").render_notebook()

In [10]:
get_line(get_data("套利"), "套利").render_notebook()

In [11]:
tab = Tab()
tab.add(table_index, "各策略CTA产品涨跌幅")
tab.add(get_line(get_data("主观CTA"), "主观CTA"), "主观CTA走势")
tab.add(get_line(get_data("量化CTA"), "量化CTA"), "量化CTA走势")
tab.add(get_line(get_data("套利"), "套利"), "套利")
tab.render("CTA策略收益跟踪.html")

'e:\\桌面文件\\Vscode\\Private_nav_research\\CTA策略收益跟踪.html'